# Neutral Sentiment A/B Test (Upvote vs FinBERT) with Robustness

Compare TFT+Reddit when neutral class is defined by Upvote ratio (A) vs FinBERT (B), and run robustness across multiple seeds (mean/std).


In [1]:
# Path setup
import os, sys
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
DATA_DIR = os.path.join(PROJECT_ROOT, 'data', 'processed')
A_PATH = os.path.join(DATA_DIR, 'tsla_price_sentiment_spike.csv')       # Upvote neutral
B_PATH = os.path.join(DATA_DIR, 'tsla_price_sentiment_spike_new.csv')  # FinBERT neutral
RESULTS_DIR = os.path.join(PROJECT_ROOT, 'results')
print('A:', A_PATH)
print('B:', B_PATH)



A: /Users/hwang-yejin/Desktop/Financial Time Series Forecasting with Deep Learning Models and Social Media Sentiment/data/processed/tsla_price_sentiment_spike.csv
B: /Users/hwang-yejin/Desktop/Financial Time Series Forecasting with Deep Learning Models and Social Media Sentiment/data/processed/tsla_price_sentiment_spike_new.csv


In [2]:
# Deterministic seed helper
import os, random, numpy as np, torch
try:
    import lightning.pytorch as pl
except Exception:
    pl = None

def set_seed(seed: int):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if pl is not None:
        try:
            pl.seed_everything(seed, workers=False)
        except Exception:
            pass
    try:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    except Exception:
        pass


In [3]:
# Runner wrappers
from importlib import reload
from models import tft_with_reddit_sentiment as tft_r
reload(tft_r)

def run_with_data_seeded(data_path: str, seed: int):
    set_seed(seed)
    config = tft_r.get_user_config()
    config['train_start'] = '2025-02-01'
    config['training_days'] = 96
    config['prediction_days'] = 5
    df = tft_r.load_and_prepare_data(data_path, config)
    training_dataset, _ = tft_r.create_tft_dataset(df, config)
    tft, train_loader = tft_r.create_model_and_dataloader(training_dataset, config)
    trainer = tft_r.train_model(tft, train_loader, training_dataset, config)
    best_tft, val_loader = tft_r.load_best_model_and_validate(trainer, training_dataset, df, tft, config)
    preds, actuals = tft_r.make_predictions(best_tft, val_loader)
    return tft_r.evaluate_performance(preds, actuals)


In [ ]:
# Override runner to be fully non-interactive (no input prompts)
from importlib import reload
from models import tft_with_reddit_sentiment as tft_r
reload(tft_r)

def run_with_data_seeded(data_path: str, seed: int):
    set_seed(seed)
    # Fixed config (no prompts)
    config = {
        'training_type': 'date_anchor',
        'train_start': '2025-02-01',
        'training_days': 96,
        'prediction_days': 5,
        'max_epochs': 30,
        'batch_size': 128,
        'learning_rate': 0.03,
    }
    df = tft_r.load_and_prepare_data(data_path, config)
    training_dataset, _ = tft_r.create_tft_dataset(df, config)
    tft, train_loader = tft_r.create_model_and_dataloader(training_dataset, config)
    trainer = tft_r.train_model(tft, train_loader, training_dataset, config)
    best_tft, val_loader = tft_r.load_best_model_and_validate(trainer, training_dataset, df, tft, config)
    preds, actuals = tft_r.make_predictions(best_tft, val_loader)
    return tft_r.evaluate_performance(preds, actuals)


In [4]:
# A/B single run (optional)
import time, pandas as pd
print('Running A (Upvote neutral) ...')
start = time.time(); A = run_with_data_seeded(A_PATH, 42); A_t = time.time()-start
print('A:', A, f'({A_t:.2f}s)')
print('\nRunning B (FinBERT neutral) ...')
start = time.time(); B = run_with_data_seeded(B_PATH, 42); B_t = time.time()-start
print('B:', B, f'({B_t:.2f}s)')

cmp = pd.DataFrame({'A Upvote': A, 'B FinBERT': B})
print('\nSingle-seed comparison:')
print(cmp)



Seed set to 42


Running A (Upvote neutral) ...

=== TFT Configuration (96-day training, 5-day prediction) ===

✓ Configuration set:
  - Training start: 2025-02-01
  - Training days: 96
  - Prediction days: 5
  - Max epochs: 30
  - Batch size: 128
  - Learning rate: 0.03

=== Loading and Preparing Data ===
✓ Data loaded successfully from /Users/hwang-yejin/Desktop/Financial Time Series Forecasting with Deep Learning Models and Social Media Sentiment/data/processed/tsla_price_sentiment_spike.csv
  - Shape: (232, 19)
  - Date range: 2024-06-04 to 2025-07-22

Data columns:
['time_idx', 'date', 'close', 'volume', 'days_since_earning', 'month', 'day_of_week', 'quarter', 'year', 'is_month_end', 'is_month_start', 'rolling_volatility', 'return_1d', 'cumulative_return', 'unique_id', 'daily_sentiment', 'post_count', 'spike_presence', 'spike_intensity', 'daily_sentiment_lag1', 'daily_sentiment_lag2', 'daily_sentiment_lag3', 'daily_sentiment_lag4', 'daily_sentiment_lag5', 'daily_sentiment_mean_3', 'daily_sentiment

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


✓ TFT model created with 50343 parameters
  - Learning rate: 0.03
  - Hidden size: 16
  - Attention heads: 1

=== Training TFT Model ===
✓ Trainer configured with 30 max epochs (monitoring train_loss)
  - Early stopping enabled
  - Learning rate monitoring enabled
  - Model checkpointing enabled

Starting model training...



   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | TorchMetricWrapper              | 0      | train
1  | logging_metrics                    | ModuleList                      | 0      | train
2  | input_embeddings                   | MultiEmbedding                  | 1      | train
3  | prescalers                         | ModuleDict                      | 432    | train
4  | static_variable_selection          | VariableSelectionNetwork        | 48     | train
5  | encoder_variable_selection         | VariableSelectionNetwork        | 19.3 K | train
6  | decoder_variable_selection         | VariableSelectionNetwork        | 17.2 K | train
7  | static_context_variable_selection  | GatedResidualNetwork            | 1.1 K  | train
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 1.1 K  

...

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric train_loss did not improve in the last 5 records. Best score: 13.806. Signaling Trainer to stop.


✅ Training completed!

=== Loading Best Model and Creating Validation Dataset ===


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


✓ Best model loaded from: /Users/hwang-yejin/Desktop/Financial Time Series Forecasting with Deep Learning Models and Social Media Sentiment/notebooks/checkpoints/tft-epoch=10-train_loss=13.8061.ckpt
✓ Validation dataset created with 1 samples

=== Making Predictions ===
✓ Predictions made successfully
  - Prediction shape: torch.Size([1, 5])
  - Actuals shape: torch.Size([1, 5])

=== Performance Evaluation ===

📊 Model Performance Metrics:
  Metric    Value
0    MAE   5.5511
1    MSE  34.0470
2   RMSE   5.8350
3   MAPE   1.7598
B: {'MAE': 5.551110744476318, 'MSE': 34.046958923339844, 'RMSE': 5.834977149963379, 'MAPE': 1.7598493099212646} (61.47s)

Single-seed comparison:
       A Upvote  B FinBERT
MAE    4.397833   5.551111
MSE   20.192596  34.046959
RMSE   4.493618   5.834977
MAPE   1.390320   1.759849


In [6]:
# Robustness: seeds mean/std
import time, pandas as pd
seeds = [42, 43, 44, 45, 46]
rows_A, rows_B = [], []
for s in seeds:
    t0 = time.time(); mA = run_with_data_seeded(A_PATH, s); dtA = time.time()-t0
    rows_A.append({"seed": s, **mA, "Time(s)": dtA})
    t0 = time.time(); mB = run_with_data_seeded(B_PATH, s); dtB = time.time()-t0
    rows_B.append({"seed": s, **mB, "Time(s)": dtB})
A_df = pd.DataFrame(rows_A).set_index('seed')
B_df = pd.DataFrame(rows_B).set_index('seed')

summary = pd.DataFrame({
    ('A Upvote','MAE mean'): [A_df['MAE'].mean()],
    ('A Upvote','MAE std'): [A_df['MAE'].std(ddof=1)],
    ('A Upvote','RMSE mean'): [A_df['RMSE'].mean()],
    ('A Upvote','RMSE std'): [A_df['RMSE'].std(ddof=1)],
    ('A Upvote','MAPE mean'): [A_df['MAPE'].mean()],
    ('A Upvote','MAPE std'): [A_df['MAPE'].std(ddof=1)],
    ('B FinBERT','MAE mean'): [B_df['MAE'].mean()],
    ('B FinBERT','MAE std'): [B_df['MAE'].std(ddof=1)],
    ('B FinBERT','RMSE mean'): [B_df['RMSE'].mean()],
    ('B FinBERT','RMSE std'): [B_df['RMSE'].std(ddof=1)],
    ('B FinBERT','MAPE mean'): [B_df['MAPE'].mean()],
    ('B FinBERT','MAPE std'): [B_df['MAPE'].std(ddof=1)],
}).T
summary.columns = ['value']

print('\nPer-seed A (Upvote):\n', A_df)
print('\nPer-seed B (FinBERT):\n', B_df)
print('\nRobustness summary (mean±std):\n', summary)

# Save
import os
A_df.to_csv(os.path.join(RESULTS_DIR, 'TSLA_neutral_ab_seeds_A_upvote.csv'))
B_df.to_csv(os.path.join(RESULTS_DIR, 'TSLA_neutral_ab_seeds_B_finbert.csv'))
summary.to_csv(os.path.join(RESULTS_DIR, 'TSLA_neutral_ab_summary.csv'))
print('Saved CSVs to', RESULTS_DIR)


Seed set to 42



=== TFT Configuration (96-day training, 5-day prediction) ===

✓ Configuration set:
  - Training start: 2025-02-01
  - Training days: 96
  - Prediction days: 5
  - Max epochs: 30
  - Batch size: 128
  - Learning rate: 0.03

=== Loading and Preparing Data ===
✓ Data loaded successfully from /Users/hwang-yejin/Desktop/Financial Time Series Forecasting with Deep Learning Models and Social Media Sentiment/data/processed/tsla_price_sentiment_spike.csv
  - Shape: (232, 19)
  - Date range: 2024-06-04 to 2025-07-22

Data columns:
['time_idx', 'date', 'close', 'volume', 'days_since_earning', 'month', 'day_of_week', 'quarter', 'year', 'is_month_end', 'is_month_start', 'rolling_volatility', 'return_1d', 'cumulative_return', 'unique_id', 'daily_sentiment', 'post_count', 'spike_presence', 'spike_intensity', 'daily_sentiment_lag1', 'daily_sentiment_lag2', 'daily_sentiment_lag3', 'daily_sentiment_lag4', 'daily_sentiment_lag5', 'daily_sentiment_mean_3', 'daily_sentiment_std_3', 'daily_sentiment_mean_

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


✓ Trainer configured with 30 max epochs (monitoring train_loss)
  - Early stopping enabled
  - Learning rate monitoring enabled
  - Model checkpointing enabled

Starting model training...



   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | TorchMetricWrapper              | 0      | train
1  | logging_metrics                    | ModuleList                      | 0      | train
2  | input_embeddings                   | MultiEmbedding                  | 1      | train
3  | prescalers                         | ModuleDict                      | 432    | train
4  | static_variable_selection          | VariableSelectionNetwork        | 48     | train
5  | encoder_variable_selection         | VariableSelectionNetwork        | 19.3 K | train
6  | decoder_variable_selection         | VariableSelectionNetwork        | 17.2 K | train
7  | static_context_variable_selection  | GatedResidualNetwork            | 1.1 K  | train
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 1.1 K  

...

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric train_loss did not improve in the last 5 records. Best score: 63.051. Signaling Trainer to stop.


✅ Training completed!

=== Loading Best Model and Creating Validation Dataset ===


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


✓ Best model loaded from: /Users/hwang-yejin/Desktop/Financial Time Series Forecasting with Deep Learning Models and Social Media Sentiment/notebooks/checkpoints/tft-epoch=05-train_loss=63.0506.ckpt
✓ Validation dataset created with 1 samples

=== Making Predictions ===
✓ Predictions made successfully
  - Prediction shape: torch.Size([1, 5])
  - Actuals shape: torch.Size([1, 5])

=== Performance Evaluation ===

📊 Model Performance Metrics:
  Metric     Value
0    MAE   20.9365
1    MSE  455.0829
2   RMSE   21.3327
3   MAPE    6.6037

Per-seed A (Upvote):
             MAE          MSE       RMSE       MAPE    Time(s)
seed                                                         
42     4.397833    20.192596   4.493618   1.390320  63.757982
43     3.458209    13.455104   3.668120   1.094604  64.680809
44     1.711438     3.861286   1.965015   0.542998  61.537073
45     1.312439     3.125810   1.767996   0.419040  70.999236
46    80.763687  6544.574707  80.898544  25.521740  22.752243

Per

In [11]:
# Show per-seed and summary results instead of global results matrix
import os, pandas as pd

A_df_path = os.path.join(RESULTS_DIR, 'TSLA_neutral_ab_seeds_A_upvote.csv')
B_df_path = os.path.join(RESULTS_DIR, 'TSLA_neutral_ab_seeds_B_finbert.csv')
summary_path = os.path.join(RESULTS_DIR, 'TSLA_neutral_ab_summary.csv')

if os.path.exists(A_df_path) and os.path.exists(B_df_path) and os.path.exists(summary_path):
    A_df = pd.read_csv(A_df_path, index_col=0)
    B_df = pd.read_csv(B_df_path, index_col=0)
    summary = pd.read_csv(summary_path, index_col=0)
    print('\nPer-seed A (Upvote):\n', A_df)
    print('\nPer-seed B (FinBERT):\n', B_df)
    print('\nRobustness summary (mean±std):\n', summary)
else:
    print('One or more result CSVs not found in:', RESULTS_DIR)



Per-seed A (Upvote):
             MAE          MSE       RMSE       MAPE    Time(s)
seed                                                         
42     4.397833    20.192596   4.493618   1.390320  63.757982
43     3.458209    13.455104   3.668120   1.094604  64.680809
44     1.711438     3.861286   1.965015   0.542998  61.537073
45     1.312439     3.125810   1.767996   0.419040  70.999236
46    80.763687  6544.574707  80.898544  25.521740  22.752243

Per-seed B (FinBERT):
             MAE         MSE       RMSE      MAPE    Time(s)
seed                                                       
42     5.551111   34.046959   5.834977  1.759849  43.465069
43     6.747394   47.713783   6.907516  2.131168  50.900401
44    14.755689  235.453537  15.344496  4.684050  23.309640
45     1.813226    5.911119   2.431279  0.576063  51.303723
46    20.936548  455.082916  21.332672  6.603738  31.432009

Robustness summary (mean±std):
       A Upvote mean  A Upvote std  B FinBERT mean  B FinBERT std
M